In [12]:

%pprint

Pretty printing has been turned OFF


In [26]:

from pandas import read_excel, to_datetime, Series

file_path = '../data/xlsx/Alignment_x_Trust_Correlations.xlsx'
alignment_x_trust_correlations_df = read_excel(file_path)
alignment_x_trust_correlations_df.columns = alignment_x_trust_correlations_df.columns.map(lambda x: '_'.join([s.title() for s in x.split('_')]))
alignment_x_trust_correlations_df.columns = alignment_x_trust_correlations_df.columns.map(lambda x: x[:2].upper() + x[2:])
alignment_x_trust_correlations_df = alignment_x_trust_correlations_df.rename(columns={'PArticipantid': 'participant_id'})
sorted(alignment_x_trust_correlations_df.columns)

['AD_Align_High_Four', 'AD_Align_High_Sim', 'AD_Align_High_Text', 'AD_Align_Low_Four', 'AD_Align_Low_Sim', 'AD_Align_Low_Text', 'AD_High_Agree', 'AD_High_Alignsr', 'AD_High_Trust', 'AD_High_Trustworthy', 'AD_Low_Agree', 'AD_Low_Alignsr', 'AD_Low_Trust', 'AD_Low_Trustworthy', 'ST_Align_High_Four', 'ST_Align_High_Sim', 'ST_Align_High_Text', 'ST_Align_Low_Four', 'ST_Align_Low_Sim', 'ST_Align_Low_Text', 'ST_High_Agree', 'ST_High_Agree_Omni', 'ST_High_Alignsr', 'ST_High_Alignsr_Omni', 'ST_High_Trust', 'ST_High_Trust_Omni', 'ST_High_Trustworthy', 'ST_High_Trustworthy_Omni', 'ST_Low_Agree', 'ST_Low_Agree_Omni', 'ST_Low_Alignsr', 'ST_Low_Alignsr_Omni', 'ST_Low_Trust', 'ST_Low_Trust_Omni', 'ST_Low_Trustworthy', 'ST_Low_Trustworthy_Omni', 'participant_id']

In [39]:

import pingouin as pg

def calculate_icc(participant_data, trust_rating_variable_name, alignment_score_variable_name, targets_column_name='participant_id'):
  """
  Calculates ICC(2,1) for a given DataFrame.

  Args:
      participant_data: The DataFrame containing participant data.
      trust_rating_variable_name: The column name for the trust rating variable.
      alignment_score_variable_name: The column name for the specific alignment score variable (e.g., AD_Align_High_Text).
      targets_column_name: The column name for participant ID. Defaults to 'participant_id'.

  Returns:
      A pingouin ICC results object.
  """
  return pg.intraclass_corr(
      data=participant_data,
      targets=targets_column_name,
      raters=trust_rating_variable_name,
      ratings=alignment_score_variable_name,
      # nan_policy='omit'
  )


I have separate columns in the alignment_x_trust_correlations_dfn DataFrame for AD High Trust, AD Low Trust, etc. The relationships I want primarily need to understand are:

 - AD_Align_High_Text by AD_High_Trust
 - AD_Align_Low_Text by AD_Low_Trust

 - ST_Align_High_Text by ST_High_Trust
 - ST_Align_Low_Text by ST_Low_Trust

In [61]:

from scipy.stats import pointbiserialr

# Loop through desired groups
relationships_str = '''AD_Align_High_Text by AD_High_Trust|AD_Align_Low_Text by AD_Low_Trust
ST_Align_High_Text by ST_High_Trust|ST_Align_Low_Text by ST_Low_Trust'''

for relationship_str in relationships_str.split('\n'):
    high_low_list = relationship_str.split('|')
    high_variable_names_list = high_low_list[0].split(' by ')
    low_variable_names_list = high_low_list[1].split(' by ')
    
    # Define column names for high trust rating and high alignment score column name
    high_trust_rating_variable_name = high_variable_names_list[1]
    high_alignment_score_column_name = high_variable_names_list[0]
    
    # Select data for the high trust group (assuming separate columns)
    high_columns_list = ['participant_id', high_alignment_score_column_name, high_trust_rating_variable_name]
    high_trust_data = alignment_x_trust_correlations_df[high_columns_list].dropna()
    
    # Define column names for low trust rating and low alignment score column name
    low_trust_rating_variable_name = low_variable_names_list[1]
    low_alignment_score_column_name = low_variable_names_list[0]
    
    # Select data for the low trust group (assuming separate columns)
    low_columns_list = ['participant_id', low_alignment_score_column_name, low_trust_rating_variable_name]
    low_trust_data = alignment_x_trust_correlations_df[low_columns_list].dropna()
    # print(
    #     high_trust_data.shape, high_trust_data[high_alignment_score_column_name].shape[0],
    #     high_trust_data[high_trust_rating_variable_name].shape[0]
    # ); continue
    
    # Calculate point-biserial correlation for high trust
    high_trust_corr, _ = pointbiserialr(high_trust_data[high_alignment_score_column_name], high_trust_data[high_trust_rating_variable_name])
    
    # Calculate point-biserial correlation for low trust
    low_trust_corr, _ = pointbiserialr(low_trust_data[low_alignment_score_column_name], low_trust_data[low_trust_rating_variable_name])
    
    # Print results for the current group
    print(f'Point-biserial Correlation ({high_low_list[0]}):')
    print(high_trust_corr)
    print(f'Point-biserial Correlation ({high_low_list[1]}):')
    print(low_trust_corr)

Point-biserial Correlation (AD_Align_High_Text by AD_High_Trust):
-0.14254941387335285
Point-biserial Correlation (AD_Align_Low_Text by AD_Low_Trust):
0.5295439763044246
Point-biserial Correlation (ST_Align_High_Text by ST_High_Trust):
0.12771726492674046
Point-biserial Correlation (ST_Align_Low_Text by ST_Low_Trust):
-0.0027825601658847295
